Making the whole model pipeline in pytorch

1. Design the model (input, output_size, fwd pass)
2. construct loss and optimizer
3. training loop

- forward pass: compute prediction
- backward pass : gradients
- update weights 

In [3]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
# prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

print(n_samples, n_features)

569 30


In [5]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

#scale the features
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

#convert to torch tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test  = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))


# making y_train as column vector as tensors will be a 2d list sort of
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)


In [6]:
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression,self).__init__()
    self.linear = nn.Linear(n_input_features, 1)
  

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    return y_predicted

#layer will be having size 30*1
model = LogisticRegression(n_features)

#cross entropy loss
loss_criterion = nn.BCELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


#training loop
num_epochs = 100

for epoch in range(num_epochs):
  #fwd pass
  y_predicted = model(X_train)

  loss = loss_criterion(y_predicted, y_train)

  #bwd propogation
  loss.backward()

  optimizer.step()

  optimizer.zero_grad()

  if (epoch+1)%10 ==0:
    print(f'epoch: {epoch+1} , loss={loss.item():.4f}')


#evaluation now our old computational graph should not be there
with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum()/ float(y_test.shape[0])
  print(f'accuracy = {acc:.4f}')

epoch: 10 , loss=0.5051
epoch: 20 , loss=0.4379
epoch: 30 , loss=0.3916
epoch: 40 , loss=0.3574
epoch: 50 , loss=0.3309
epoch: 60 , loss=0.3097
epoch: 70 , loss=0.2923
epoch: 80 , loss=0.2776
epoch: 90 , loss=0.2651
epoch: 100 , loss=0.2542
accuracy = 0.9474
